In [45]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score
import numpy as np


In [46]:
# Define transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validate': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets using ImageFolder
data_dir='devide3.0'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'validate','test']}

# Wrap datasets in DataLoader
dataloaders = {x: DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=4)
               for x in ['train', 'validate','test']}

# Get dataset sizes and class names
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'validate','test']}
class_names = image_datasets['train'].classes
num_classes = len(class_names)

# Load a pre-trained VGG11 model
model = models.vgg11(pretrained=True)

# Freeze parameters so we don't backprop through them (optional)
for param in model.features.parameters():
    param.requires_grad = False

# Modify the classifier to match the number of classes
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

# Clear any unused memory
torch.cuda.empty_cache()

# Move model to the CUDA device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)


c:\Users\jake Hu\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jake Hu\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Using device: cuda:0


In [47]:
# Train the model
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'validate']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        print()

    return model

In [48]:
model_ft = train_model(model, criterion, optimizer, num_epochs=15)

Epoch 0/14
----------
train Loss: 1.6740 Acc: 0.4547
validate Loss: 1.2296 Acc: 0.4633

Epoch 1/14
----------
train Loss: 1.6225 Acc: 0.4851
validate Loss: 0.9903 Acc: 0.7143

Epoch 2/14
----------
train Loss: 1.5562 Acc: 0.5207
validate Loss: 1.0468 Acc: 0.5985

Epoch 3/14
----------
train Loss: 1.3921 Acc: 0.5243
validate Loss: 0.8476 Acc: 0.6911

Epoch 4/14
----------
train Loss: 1.3845 Acc: 0.5233
validate Loss: 0.8494 Acc: 0.6834

Epoch 5/14
----------
train Loss: 1.2778 Acc: 0.5492
validate Loss: 0.8707 Acc: 0.6950

Epoch 6/14
----------
train Loss: 1.2491 Acc: 0.5466
validate Loss: 0.9187 Acc: 0.6486

Epoch 7/14
----------
train Loss: 1.2054 Acc: 0.5625
validate Loss: 0.9897 Acc: 0.5869

Epoch 8/14
----------
train Loss: 1.1904 Acc: 0.5741
validate Loss: 0.8693 Acc: 0.6834

Epoch 9/14
----------
train Loss: 1.1971 Acc: 0.5652
validate Loss: 0.8647 Acc: 0.6911

Epoch 10/14
----------
train Loss: 1.1513 Acc: 0.5782
validate Loss: 0.8342 Acc: 0.7259

Epoch 11/14
----------
train Lo

In [49]:
# Evaluate the model
def evaluate_model(model, dataloaders, phase='test'):
    model.eval()   # Set the model to evaluation mode
    all_preds = []
    all_labels = []
    
    # Disabling gradient calculation
    with torch.no_grad():
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate accuracy and other metrics using sklearn
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    return accuracy, f1

# Get test accuracy and F1 score
test_accuracy, test_f1 = evaluate_model(model_ft, dataloaders, 'test')
print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'Test F1 Score: {test_f1:.4f}')



Test Accuracy: 0.7100
Test F1 Score: 0.6426
